# Universidad del Desarrollo
## Magister Data Science - Macroeconomía
### Grupo N°2: (Región de los Lagos)
* Jose Pedro
* Cristian
* Paul
* Max
* Hernan


# Imputación de los datos

In [21]:
import pandas as pd
import seaborn as sns
from scipy.stats.mstats import gmean

In [22]:
df = pd.read_csv('output/rlagos_canasta.csv')

In [23]:
df.head()

,Calidad,Mes/Año,Momento,Precio máximo,Precio mínimo,Precio promedio,Procedencia,Producto,Sector,Tipo,Tipo punto monitoreo,Unidad,Variedad
0,NaN,08/2014,2014-08,2249,1129,1558.2966,NaN,Maravilla,Osorno,Aceite,NaN,$/litro,NaN
1,NaN,08/2014,2014-08,1735,790,1410.9042,NaN,Vegetal,Osorno,Aceite,NaN,$/litro,NaN
2,NaN,08/2014,2014-08,1719,1220,1516.6203,NaN,Maravilla,Puerto Montt,Aceite,NaN,$/litro,NaN
3,NaN,08/2014,2014-08,1749,809,1357.3113,NaN,Vegetal,Puerto Montt,Aceite,NaN,$/litro,NaN
4,NaN,09/2014,2014-09,1768,1589,1680.2222,NaN,Maravilla,Frutillar,Aceite,NaN,$/litro,NaN


In [24]:
df.shape

(32362, 13)

### Paso 1: buscar todos los promedios geometricos

Creamos un dataset nuevo con todas las combinaciones Tipo / Sector

In [26]:
Combos = df.groupby(['Tipo','Producto','Sector']).count().reset_index()[['Tipo','Producto','Sector']]

In [27]:
Combos.head()

,Tipo,Producto,Sector
0,Aceite,Maravilla,Frutillar
1,Aceite,Maravilla,Llanquihue
2,Aceite,Maravilla,Osorno
3,Aceite,Maravilla,Puerto Montt
4,Aceite,Maravilla,Puerto Varas


Una prueba para ver como calculamos el promedio geometrico de una combinación a mano, antes de crear una función:

In [28]:
gmean(df[(df.Tipo == 'Aceite') & (df.Producto == 'Maravilla') & (df.Sector == 'Osorno')]['Precio promedio'])

1619.1497080362053

Creamos una función que calcula el promedio geometrico en base al dataframe completo, pero filtrado por el dataframe Tipo / Producto / Sector

In [29]:
def calc_gmean(row):
    Tipo = row['Tipo']
    Producto = row['Producto']
    Sector = row['Sector']
    a = gmean(df[(df.Tipo == Tipo) & (df.Producto == Producto) & (df.Sector == Sector)]['Precio promedio'])
    return a

Aplicamos la función a cada fila del dataframe con las combinaciones Tipo / Producto / Sector

In [30]:
Combos['PromGeo'] = Combos.apply(calc_gmean, axis=1)

In [31]:
Combos.sample(10)

,Tipo,Producto,Sector,PromGeo
540,Pan,Hallulla integral,Frutillar,1397.600525
248,Frutas,Durazno,Osorno,1225.912745
28,Carnes,Asado de tira,Puerto Montt,5084.832992
130,Carnes,Pavo Trutro Corto,Osorno,2904.070973
545,Pan,Marraqueta,Frutillar,1111.210718
150,Carnes,Pollo Pechuga,Osorno,2382.807724
362,Hortaliza,Coliflor,Puerto Montt,964.268860
410,Hortaliza,Repollo,Frutillar,1205.594772
149,Carnes,Pollo Pechuga,Llanquihue,2640.879941
405,Hortaliza,Poroto verde,Frutillar,2336.873844


### Paso 2: Rellenar lo faltante

Estrategia:

1. Crear un dataframe con todas las fechas (momentos) del periodo.
1. A partir de cada columna en el dataframe *Combos*, crear un dataframe de una combinación Tipo/Producto/Sector
1. Hacer un merge entre el dataset de los momentos y el dataset Tipo/Producto/Sector para ver qué son los *momentos* que no tienen valores
1. Llenar los NaN con el promedio geometrico (y otros datos faltantes dado que estas filas no existían antes)
1. Crear un nuevo dataframe donde vamos agregando los dataframes de Tipo/Producto/Sector a medida que se completen.

In [32]:
df_momentos = pd.DataFrame(df['Momento'])
df_momentos.drop_duplicates(subset='Momento', keep='first', inplace=True)
df_momentos.reset_index()
df_momentos.shape

(47, 1)

In [33]:
df_final = pd.DataFrame([])

def fill_nans(row):
    global df_final
    Tipo = row['Tipo']
    Producto = row['Producto']
    Sector = row['Sector']
    PromGeo = row['PromGeo']
    # Creamos un dataset con la combinación
    data = df[(df.Tipo == Tipo) & (df.Producto == Producto) & (df.Sector == Sector)][['Momento','Precio promedio','Tipo','Producto','Sector']]
    # Hacemos merge con el dataset que contiene todos los momentos
    data = pd.merge(df_momentos, data, on='Momento', how='left')
    # Llenamos los NaN
    data['Precio promedio'].fillna(PromGeo, inplace=True)
    data['Tipo'].fillna(Tipo, inplace=True)
    data['Producto'].fillna(Producto, inplace=True)
    data['Sector'].fillna(Sector, inplace=True)
    # Agregamos la data a al dataframe completo
    df_final = df_final.append(data)
    pass

In [34]:
Combos.apply(fill_nans, axis=1)
df_final.head()

,Momento,Precio promedio,Tipo,Producto,Sector
0,2014-08,1807.484158,Aceite,Maravilla,Frutillar
1,2014-09,1680.222200,Aceite,Maravilla,Frutillar
2,2014-10,1651.904700,Aceite,Maravilla,Frutillar
3,2014-11,1807.484158,Aceite,Maravilla,Frutillar
4,2014-12,1807.484158,Aceite,Maravilla,Frutillar


In [35]:
df_final.shape

(38500, 5)

In [36]:
df_final.reset_index(inplace=True)

Test con los que sabemos que estaban missing:

In [37]:
df_final[(df_final.Tipo == 'Hortaliza') & (df_final.Producto == 'Repollo') & (df_final.Sector == 'Frutillar') & (df_final.Momento == '2014-08')]

,index,Momento,Precio promedio,Tipo,Producto,Sector
29741,0,2014-08,1205.594772,Hortaliza,Repollo,Frutillar


In [38]:
dx = df_final.groupby(["Momento", "Sector"])['Producto'].size().unstack()

mycolor=['black', 'red', 'green', 'blue', 'cyan']

dx.plot(kind='bar', stacked=True, figsize=(20, 6), color=mycolor, rot=45)

In [39]:
df = df_final[["Momento", "Sector", "Tipo", "Producto", "Precio promedio"]]

In [40]:
df.rename(columns={'Producto': 'Variedad', 'Precio promedio': 'Promedio', 'Tipo': 'Producto'}, inplace=True)

/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [41]:
df.to_csv('output/canasta_ipc.csv', index=False)